<a href="https://colab.research.google.com/github/aljebraschool/ai-startup-idea-generator/blob/master/LLM_university_Introduction_to_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install --upgrade cohere -q

In [25]:
import cohere

In [26]:
co = cohere.ClientV2("COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

# Define the Documents

Next, we define the documents that we want to ground an LLM’s response with, formatted as a list. In our case, each document consists of two fields: title and text.

The documents list includes a list of documents with a “text” field containing the information we want the model to use. The recommended length for the snippet of each document is relatively short, 300 words or less. We recommend using field names similar to the ones we’ve included in this example (i.e., “title” and “text”), but RAG is quite flexible with respect to how you structure the documents listings. You can give the fields any names you want, and you can pass in other fields as well, such as a “date” field. All field names and field values are passed to the model.

In [36]:
documents = [
    {"title": "Who is Ridwan Ibidunni",
     "text": "Ridwan is a mathematician with several years of experience programming and solving problems using code"},
    {"title": "Programming languages Ridwan loves",
     "text": "He loves coding in Java, Python and using machine learning frameworks like PyTorch, Tensorflow and scikit-learn"}

  ]


# Generate the Response with Citations

Cohere’s RAG functionalities are part of the Chat endpoint, with the Command model as the underlying LLM. This allows developers to build chatbots that have the full context of a conversation and are not limited to a single interaction.

First, we define the message coming from the user. We’ll use a simple query,

Then, we pass this message as a message parameter to a Chat endpoint call. We also pass the list of documents as a documents parameter. By using the chat_stream method, the response is generated incrementally by token without having to wait for the full completion.

Finally, we display the response from the model. The streamed response will return different types of objects, and for now, we are interested in the text-generation objects, which contain the generated text.

We also display the citations and source documents, which we can get from the final object returned by the streamed response.

In [42]:
# Get the user message
prompt = "Does he loves maths"

# Generate the response
response = co.chat_stream(
    message = prompt,
    documents=documents,
    model="command-r-plus",
)


#display the response
citations = []
cited_document = []

for event in response:
    if event.event_type == "text-generation":
        print(event.text, end="")
    elif event.event_type == "citation-generation":
        citations.extend(event.citations)
    elif event.event_type == "stream-end":
      cited_documents = event.response.documents

# Display the citations and source documents
if citations:
  print("\n\nCITATIONS:")
  for citation in citations:
    print(citation)

  print("\nDOCUMENTS:")
  for document in cited_documents:
    print(document)

Yes, Ridwan Ibidunni is a mathematician with several years of experience programming and solving problems using code. He loves coding in Java and Python and using machine learning frameworks like PyTorch, Tensorflow and scikit-learn.

CITATIONS:
start=5 end=20 text='Ridwan Ibidunni' document_ids=['doc_0']
start=26 end=39 text='mathematician' document_ids=['doc_0']
start=45 end=117 text='several years of experience programming and solving problems using code.' document_ids=['doc_0']
start=127 end=152 text='coding in Java and Python' document_ids=['doc_1']
start=157 end=233 text='using machine learning frameworks like PyTorch, Tensorflow and scikit-learn.' document_ids=['doc_1']

DOCUMENTS:
{'id': 'doc_0', 'text': 'Ridwan is a mathematician with several years of experience programming and solving problems using code', 'title': 'Who is Ridwan Ibidunni'}
{'id': 'doc_1', 'text': 'He loves coding in Java, Python and using machine learning frameworks like PyTorch, Tensorflow and scikit-learn'